In [4]:
import xml.etree.ElementTree as ET

In [5]:
tree = ET.parse('../../Data/Datasets/Common/suc3testset.xml')
root = tree.getroot()
ents = ["PRS", "ORG", "LOC", "TME"]
sentsList = []

for sent in root.iter('sentence'):
    prevEntType = None
    tokensList = []
    for child in sent:
        if child.tag == 'name':
            prevEntType = None
            for token in child.iter('w'):
                nameDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                tokensList.append(nameDict)
        elif child.tag == 'ne' and child.attrib['type'] in ents and child.attrib['type'] == prevEntType:
            entityTokenList = child.attrib['name'].split(' ')
            for token in child.iter('w'):
                neDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|')}
                if token.text == entityTokenList[0]:
                    neDict['ner'] = 'B-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
                else:
                    neDict['ner'] = 'I-' + child.attrib['type']
                    tokensList.append(neDict)
                    continue
        elif child.tag == 'ne' and child.attrib['type'] in ents:
            prevEntType = child.attrib['type']
            for token in child.iter('w'):
                neDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), 'ner': 'I-' + child.attrib['type']}
                tokensList.append(neDict)
        elif child.tag == 'ne':
            prevEntType = None
            for token in child.iter('w'):
                tokenDict = {'orth': token.text, 'tag': token.attrib['msd'].replace('.', '|'), "ner": "O"}
                tokensList.append(tokenDict)
        elif child.tag == 'w':
            prevEntType = None
            tokenDict = {'orth': child.text, 'tag': child.attrib['msd'].replace('.', '|'), "ner": "O"}
            tokensList.append(tokenDict)
        else:
            prevEntType = None
            continue
    sentsList.append(tokensList)

In [6]:
with open('../../Data/Datasets/ner/SUC3testCONLL2003', 'w', encoding='utf-8') as f:
    f.write('-DOCSTART- -X- O O' + '\n\n')
    for sent in sentsList:
        for token in sent:
            f.write(token['orth'] + ' ' + token['tag'] + ' ' + 'O' + ' ' + token['ner'] + '\n')
        f.write('\n')

